# Import Packages

In [1]:
import pandas as pd
import numpy as np
import os # 20250406 George

# Set Folders

In [2]:
# Get current working directory

current_dir = os.getcwd()  # 20250403 George added

# code_dir= "C:\\Users" +'\\' + username + '\\' + "Equitable Bank\\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\\RWA\\code" # 20250403 George commented out Francesca/Joseph's code

code_dir = current_dir # 20250406 George added

snapshot_num = input('Enter the snapshot needed in the format YYYYMM, e.g. 202212: ') # 20250406 George added

if int(snapshot_num) == 202412:
    input_dir = code_dir + '\\..\\' + "Inputs" # 20250406 George added
    Output_dir = code_dir + '\\..\\' + "Outputs" #20250406 George added

if int(snapshot_num) == 202212:
    input_dir = code_dir + '\\..\\' + "Dec. 2022 RWA Inputs" #20250410 George added
    Output_dir = code_dir + '\\..\\' + "Replicated Dec 2022 Outputs" #20250410 George added

In [3]:
print(Output_dir) # 20250406 George added

c:\Users\georgez\OneDrive - Equitable Bank\MD_RWA_Impact_Analysis (Joseph)\SFR\code\..\Replicated Dec 2022 Outputs


# Load Data

In [4]:
# read parquet file

eqb_rwa = pd.read_parquet(Output_dir + '\\eqb_rwa_original.parquet', engine='pyarrow') # 20250406 George added

In [5]:
# read parquet file

eqb_rwa_addon_zero = pd.read_parquet(Output_dir + '\\eqb_rwa_addon_zero.parquet', engine='pyarrow') # 20250406 George added

In [6]:
eqb_rwa.columns.values

array(['Loan_Number', 'Insured_class', 'EAD', 'Advance_Amount',
       'Years_to_maturity', 'corr_uninsured', 'RWA_standardized',
       'SL_Date', 'CalibratedPD', 'MRS_Bin_PD', 'Insured_Ind',
       'Alt_Prime_Indicator', 'RemainingPrincipal_Tot_Exp',
       'RemainingPrincipal_Excl_Partner', 'Corporate_Applicant_Ind',
       'Combo_Province_Metro_Override',
       'Combo_Province_Metro_Override_WOE',
       'Prior_24_Worse_Delinquent_Status_FMT_Adj',
       'Prior_24_Worse_Delinquent_Status_FMT_Adj_WOE',
       'Beacon_Avg_App_CoApp', 'Beacon_Avg_App_CoApp_WOE',
       'BF_LTV_Tot_Exp_FSA_Dw_WF', 'Combo_LTV_Insured_Ind',
       'Combo_LTV_Insured_Ind_WOE', 'merge_Chenxi_PD', 'MRS_Bin_LGD',
       'LR_Avg_LGD', 'Segment_Avg_LGD', 'Base_Line_LGD',
       'LGD_DT_Adjusted', 'Pre_final_LGD', 'Sub_Product',
       'BF_Appr_Prov_Dw', 'Appraisal_Bin_WOE', 'Occupancy',
       'Occupancy_WOE', 'Province_Foreclosure',
       'Province_Foreclosure_WOE', 'BF_LTV_Incl_Parter_Incl_HELOC_FSA_Dw',
 

In [12]:
eqb_rwa.Insured_Ind.value_counts()

Not Insured    30700
Insured        27894
Name: Insured_Ind, dtype: int64

# Calculate Addon for Not Insured

In [13]:
summ=eqb_rwa.query("Insured_Ind=='Not Insured'").groupby(['Combo_LTV_Insured_Ind','Combo_Province_Metro_Override_WOE'],dropna=False).RWA_new.agg(['count','sum'])

In [14]:
summ_addon_zero=eqb_rwa_addon_zero.query("Insured_Ind=='Not Insured'").groupby(['Combo_LTV_Insured_Ind','Combo_Province_Metro_Override_WOE'],dropna=False).RWA_new.agg(['count','sum']).rename(columns={'count':'count_addon_zero','sum':'sum_addon_zero'})

In [15]:
add_on_impact = pd.concat([summ,summ_addon_zero],axis=1)

In [16]:
add_on_impact['impact']=add_on_impact['sum']/add_on_impact['sum_addon_zero']

In [17]:
add_on_impact.reset_index(inplace=True)

# Sanity Check 

In [18]:
# print (add_on_impact)

print(add_on_impact)

        Combo_LTV_Insured_Ind  Combo_Province_Metro_Override_WOE  count  \
0                                                      -0.389936     21   
1                                                      -0.161658      1   
2                                                       0.165650      6   
3   Uninsured, 0.30<LTV<=0.50                          -0.389936    609   
4   Uninsured, 0.30<LTV<=0.50                          -0.161658   1047   
5   Uninsured, 0.30<LTV<=0.50                           0.165650   5224   
6   Uninsured, 0.30<LTV<=0.50                           0.253031    629   
7   Uninsured, 0.50<LTV<=0.65                          -0.389936   1582   
8   Uninsured, 0.50<LTV<=0.65                          -0.161658   1315   
9   Uninsured, 0.50<LTV<=0.65                           0.165650   4541   
10  Uninsured, 0.50<LTV<=0.65                           0.253031    765   
11  Uninsured, 0.65<LTV<=0.70                          -0.389936    628   
12  Uninsured, 0.65<LTV<=

# Save Data

In [19]:
# Save Data
#add_on_impact.to_excel(r'C:\\Users\\jasonz\\OneDrive - Equitable Bank\\Fit_for_Use Development\\RWA\\addon_impact.xlsx')
# add_on_impact.to_excel(r'C:\\Users\\francescav\\OneDrive - Equitable Bank\\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\\RWA\code\\Outputs\\\addon_impact.xlsx') # 20250406 George commented out Francesca's code

add_on_impact.to_excel(Output_dir + '\\addon_impact.xlsx') # 20250406 George added